In [15]:
#출처 : https://oboki.net/workspace/programming/language/python/folium-%EC%A7%80%EB%A6%AC%EC%A0%95%EB%B3%B4-%EC%8B%9C%EA%B0%81%ED%99%94/

### GIS 시각화

In [1]:
import cx_Oracle as oci
import pandas as pd
import numpy as np
import os
os.environ["NLS_LANG"] = ".AL32UTF8"      #encodring이 UTF-8 인 경우

# data 불러오기
# df = pd.read_csv('./report/190718_COUPON_CUST.csv', encoding = 'cp949', low_memory=False)
# GIS = df[['LAT', 'LON']].dropna()


# 접속정보
con_text = open("C:/conn_info.txt", 'r', encoding='utf8')
con_text = con_text.read()

# DB 접속
conn = oci.connect(con_text, encoding='UTF-8', nencoding='UTF-8')

query = '''
with target_group as (
    select AD.unfy_mmb_no, AD.zip_base_addr, i.wrls_tel_no1 || i.wrls_tel_no2 || i.wrls_tel_no3 as cellno, AD.psn_y_cord as lat, AD.psn_x_cord as LON
    from tm_mmb_addr AD, tm_mktg_recv_agrm A, tm_mmb_info I
    where AD.psn_y_cord is not null
    and AD.rozip_base_addr like '%서울%'
    and AD.unfy_mmb_no = A.unfy_mmb_no
    and I.unfy_mmb_no = A.unfy_mmb_no
    and A.coopco_cd = 7000 and a.sms_recv_dv_cd= 1
    and i.wrls_tel_no1 = '010'
    and I.mmb_st_cd = '1'
    and AD.psn_y_cord  between 37.4969 and 37.5097
    and AD.psn_x_cord  between 127.0252 and 127.0536
    and (2022 - (substr(I.btdy,1,4))) between 28 and 59
    )
select distinct A.* from target_group A
        '''
df = pd.read_sql(query , con = conn)

In [2]:
df1 = pd.DataFrame.to_json(df)

In [3]:
len(df)

709

In [4]:
import folium
from folium.plugins import MarkerCluster
 
# map 처음 표기위치 설정
map1 = folium.Map(location=[37.5030426, 127.041588],zoom_start=11) # 서울
# map1 = folium.Map(location=[36.5053542,127.7043419],zoom_start=8) # 한국

# 지도에 각각의 지역 표기
marker_cluster = MarkerCluster().add_to(map1)

for i in range(len(df)):
    folium.Marker(
    location=[df['LAT'].iloc[i] ,df['LON'].iloc[i]],
    icon=folium.Icon(color='red',icon='ok'),
  ).add_to(marker_cluster)

In [5]:
# 지도 표기
map1

In [12]:
# 지도 저장 : html
map1.save('map.html')

### GIS 시각화 : 특정 지역정보 기준

In [72]:
import cx_Oracle as oci
import pandas as pd
import numpy as np
import os
os.environ["NLS_LANG"] = ".AL32UTF8"      #encodring이 UTF-8 인 경우

# data 불러오기
# df = pd.read_csv('./report/190718_COUPON_CUST.csv', encoding = 'cp949', low_memory=False)
# GIS = df[['LAT', 'LON']].dropna()


# 접속정보
con_text = open("C:/conn_info.txt", 'r', encoding='utf8')
con_text = con_text.read()

# DB 접속
conn = oci.connect(con_text, encoding='UTF-8', nencoding='UTF-8')

query = '''
with target_group as (
    select AD.unfy_mmb_no
    , AD.zip_base_addr
    , REGEXP_SUBSTR(AD.rozip_base_addr, '[^ ]+',1,2) as zip_base_gubun
    , i.wrls_tel_no1 || i.wrls_tel_no2 || i.wrls_tel_no3 as cellno, AD.psn_y_cord as lat, AD.psn_x_cord as LON
    from tm_mmb_addr AD, tm_mktg_recv_agrm A, tm_mmb_info I
    where AD.psn_y_cord is not null
    and AD.rozip_base_addr  like '%대구%'
    and REGEXP_SUBSTR(AD.rozip_base_addr, '[^ ]+',1,2) in ( '동구', '중구', '수성구')  
    and AD.unfy_mmb_no = A.unfy_mmb_no
    and I.unfy_mmb_no = A.unfy_mmb_no
    and A.coopco_cd = 7000 and a.sms_recv_dv_cd= 1
    and i.wrls_tel_no1 = '010'
    and I.mmb_st_cd = '1'
    and (2022 - (substr(I.btdy,1,4))) between 35 and 45
    )
select distinct A.* from target_group A, tch_app_device_info B
where a.unfy_mmb_no = B.unfy_mmbno
        '''
df = pd.read_sql(query , con = conn)

In [73]:
df.groupby('ZIP_BASE_GUBUN')['UNFY_MMB_NO'].count()

ZIP_BASE_GUBUN
동구     404
수성구    261
중구      75
Name: UNFY_MMB_NO, dtype: int64

In [74]:
# df = df.sample(frac = 0.3)

In [75]:
len(df)

740

In [76]:
import folium
from folium.plugins import MarkerCluster
 
# map 처음 표기위치 설정
map1 = folium.Map(location=[37.5030426, 127.041588],zoom_start=11) # 서울
# map1 = folium.Map(location=[36.5053542,127.7043419],zoom_start=8) # 한국

# 지도에 각각의 지역 표기
marker_cluster = MarkerCluster().add_to(map1)

for i in range(len(df)):
    folium.Marker(
    location=[df['LAT'].iloc[i] ,df['LON'].iloc[i]],
    icon=folium.Icon(color='red',icon='ok'),
  ).add_to(marker_cluster)

In [77]:
# 지도 표기
map1

In [2]:
import folium
from folium.plugins import MarkerCluster

In [7]:
map1 = folium.Map(
  location=[37.5444042, 127.130942],
  zoom_start=15
)
 
lotto = [
  {"store":"담배1","loc":[37.5444042, 127.130942]}, 
  {"store":"담배2","loc":[37.5334158, 127.1137918]}, 
    
]
 
from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster().add_to(map1)
 
for i in range(len(lotto)):
    folium.Marker(
        location=lotto[i]['loc'],
        popup=lotto[i]['store'],
        icon=folium.Icon(color='red',icon='ok'),
    ).add_to(marker_cluster)


map1

In [3]:
y = datas_geo.geo_y.values
x = datas_geo.geo_x.value
location = list(zip(y,x))
popups = list(datas_geo.title.valuse)

icon_create_function = """\
function(cluster) {
    retuen L.divIcon({
    html: '<b>' + cluster.getChildCount() + '</b>',
    className : 'maker-cluster maker-cluster-large',
    iconSize : new L.Point(30,30)
    });
}"""


NameError: name 'datas_geo' is not defined

In [5]:
import folium
 
map1 = folium.Map(
  location=[36.5053542,127.7043419],
  zoom_start=8
)
 
lotto = [
  {"store":"담배","loc":[37.62585944357624,127.01847823823798]},   {"store":"화곡본마트","loc":[37.54248103738589,126.84414659211494]},   {"store":"용꿈돼지꿈","loc":[37.5447438346992,126.95223862043447]},   {"store":"일이오마켓","loc":[37.47943673430032,126.98346178441342]},   {"store":"여명슈퍼마켓","loc":[37.61432815661055,127.0415039221072]},   {"store":"5가로또레드탑","loc":[37.570756251987575,127.00235901546226]},   {"store":"복권세상","loc":[35.91885025656432,128.55034430109828]},   {"store":"복권전문점","loc":[37.50384327747725,126.71513104367074]},   {"store":"인현동지하가판","loc":[37.47627910055104,126.63147671205866]},   {"store":"CU(광주서동점)","loc":[35.14814807640179,126.90555219336346]},   {"store":"지산로또방","loc":[35.148338888178,126.93203920504686]},   {"store":"천사로또방","loc":[37.63618838692431,127.21107884503209]},   {"store":"오렌지통신","loc":[37.42889361993242,127.10232604117991]},   {"store":"여수복권방","loc":[37.418353650942116,127.12640523182601]},   {"store":"둘리복권방","loc":[37.34532813934063,126.73627994929885]},   {"store":"금성24시편의점","loc":[37.84401249737699,127.06236893072152]},   {"store":"GS25(청주주은점)","loc":[36.60987111587086,127.49134026856717]},   {"store":"장미슈퍼","loc":[36.276304155315685,126.90940713047799]},   {"store":"장미슈퍼","loc":[36.276304155315685,126.90940713047799]},   {"store":"아이24(수송점)","loc":[35.970108052787594,126.7189421934312]},   {"store":"탑로또","loc":[34.87844231008963,128.62803523643754]} 
]
 
from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster().add_to(map1)
 
for i in range(len(lotto)):
    folium.Marker(
        location=lotto[i]['loc'],
        popup=lotto[i]['store'],
        icon=folium.Icon(color='red',icon='ok'),
    ).add_to(marker_cluster)


In [2]:
lotto

[{'store': '담배', 'loc': [37.62585944357624, 127.01847823823798]},
 {'store': '화곡본마트', 'loc': [37.54248103738589, 126.84414659211494]},
 {'store': '용꿈돼지꿈', 'loc': [37.5447438346992, 126.95223862043447]},
 {'store': '일이오마켓', 'loc': [37.47943673430032, 126.98346178441342]},
 {'store': '여명슈퍼마켓', 'loc': [37.61432815661055, 127.0415039221072]},
 {'store': '5가로또레드탑', 'loc': [37.570756251987575, 127.00235901546226]},
 {'store': '복권세상', 'loc': [35.91885025656432, 128.55034430109828]},
 {'store': '복권전문점', 'loc': [37.50384327747725, 126.71513104367074]},
 {'store': '인현동지하가판', 'loc': [37.47627910055104, 126.63147671205866]},
 {'store': 'CU(광주서동점)', 'loc': [35.14814807640179, 126.90555219336346]},
 {'store': '지산로또방', 'loc': [35.148338888178, 126.93203920504686]},
 {'store': '천사로또방', 'loc': [37.63618838692431, 127.21107884503209]},
 {'store': '오렌지통신', 'loc': [37.42889361993242, 127.10232604117991]},
 {'store': '여수복권방', 'loc': [37.418353650942116, 127.12640523182601]},
 {'store': '둘리복권방', 'loc': [37.

In [6]:
# 지도 표기
map1